In [ ]:
# !pip install langchain_community
# !pip install pypdf
# !pip install fastembed
# !pip install chromadb
# !pip install langchain_ollama

  Using cached ollama-0.4.7-py3-none-any.whl.metadata (4.7 kB)
Using cached ollama-0.4.7-py3-none-any.whl (13 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from groq import Groq
from openai import OpenAI
import os

In [ ]:
def ingest():
    # Get the doc
    loader = PyPDFLoader("myABL User Manual.pdf")
    pages = loader.load_and_split()
    # Split the pages by char
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=700,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
        separators=["\n\n"],
    )

    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")
    #
    embedding = FastEmbedEmbeddings()
    #Create vector store
    Chroma.from_documents(documents=chunks,  embedding=embedding, persist_directory="./sql_chroma_db")

ingest() #run only first time

Split 168 documents into 212 chunks.


In [ ]:
client = Groq(api_key="")

# Call Groq API
def call_llm(prompt):
    chat_completion = client.chat.completions.create(
        # model="llama-3.3-70b-versatile",
        # model="gpt-4o-mini",
        # model="qwen-2.5-32b",
        # model="gemma2-9b-it",
        # model="allam-2-7b",
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a fast and efficient assistant."},
            {"role": "user", "content": str(prompt)},
        ],
    )
    return chat_completion.choices[0].message.content

In [9]:
from langchain_ollama import ChatOllama

def rag_chain():
    #model = ChatOllama(model="llama3.2",temperalture=0)
    model = ChatOllama(model="llama3.2:latest", temperalture=0)
    prompt = PromptTemplate.from_template(
        """
        <s> [Instructions] You are a friendly assistant. Answer the question based only on the following context. 
        If you don't know the answer, then reply, No Context availabel for this question {input}. [/Instructions] </s> 
        [Instructions] Question: {input} 
        Context: {context} 
        Answer: [/Instructions]
        """
    )
    #Load vector store
    embedding = FastEmbedEmbeddings()
    vector_store = Chroma(persist_directory="./sql_chroma_db", embedding_function=embedding)

    #Create chain
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.5,
        },
    )

    document_chain = create_stuff_documents_chain(call_llm, prompt)
    chain = create_retrieval_chain(retriever, document_chain)

    return chain

def ask(query: str):
    chain = rag_chain()
    result = chain.invoke({"input": query})
    print(result["answer"])
    # for doc in result["context"]:
    #     print("Source: ", doc.metadata["source"])

In [12]:
ask("how to change password")

To change your password, follow these steps:

1. Navigate to the icon of "More."
2. Expand the "Preferences" menu and select "Security & Settings."
3. Tap on "Change Password."
4. Enter your current password to verify your identity.
5. Upon confirmation, a screen to fill in password details will be displayed.
6. After OTP verification, enter your new password as per the policy and tap on "Submit."
